In [9]:
import requests
import json
import dml
import prov.model
import datetime
import uuid
import itertools

In [10]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = [contributor + '.famous_people', contributor + '.street_book']
writes = [contributor + '.filtered_famous_people_streets']

## exec

In [11]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [12]:
famous_people = repo[reads[0]].find()
street_book = repo[reads[1]].find()
print(famous_people)
print(street_book)

In [13]:
fp = set(itertools.chain(*{(n['first_name'], n['last_name']) for n in famous_people}))
sb = [t for t in street_book if t['Street Name'] not in fp]
print(sb)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
repo.dropCollection('filtered_famous_people_streets')
repo.createCollection('filtered_famous_people_streets')
repo[writes[0]].insert_many(sb)

In [15]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 3, 8, 21, 2, 57, 334779),
 'end': datetime.datetime(2019, 3, 8, 21, 3, 9, 727671)}

## prov

In [16]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#filtered_famous_people_streets', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
res_fp = doc.entity('bdp:fp', {'prov:label':'Famous People in MA', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
res_sb = doc.entity('bdp:sb', {'prov:label':'Street Book', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
filter_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(filter_names, this_script)
doc.usage(filter_names, res_fp, startTime, None,
    {prov.model.PROV_TYPE: 'ont:Computation',
    'ont:Computation':'Selection, Differentiate'
    }
)
doc.usage(filter_names, res_sb, startTime, None,
    {prov.model.PROV_TYPE: 'ont:Computation',
    'ont:Computation':'Selection, Differentiate'
    }
)
result = doc.entity('dat:'+contributor+'#filtered_famous_people_streets', {prov.model.PROV_LABEL:'Streets without Famous People', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(result, this_script)
doc.wasGeneratedBy(result, filter_names, endTime)
doc.wasDerivedFrom(result, res_fp, filter_names, filter_names, filter_names)
doc.wasDerivedFrom(result, res_sb, filter_names, filter_names, filter_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [17]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#filtered_famous_people_streets, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:fp, [prov:label="Famous People in MA", prov:type="ont:DataResource", ont:Extension="json"])
  entity(bdp:sb, [prov:label="Street Book", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuida4d249cb-3829-4884-b1b2-b0262cb0e6d3, -, -)
  wasAssociatedWith(log:uuida4d249cb-3829-4884-b1b2-b0262cb0e6d3, alg:mmao95_Dongyihe_weijiang_zhukk#filtered_famous_people_streets, -)
  used(log:uuida4d249cb-3829-4884-b1b2-b0262cb0e6d3, bdp:fp, -, [prov:type="ont:Computation", ont:Computation="Selection, Differentiate"])
  used(log:uuida4d249cb-3829-4884-b1b2-b0262cb0e6d3, bdp:sb, -, [prov:type="on